# AURA V12 CERBERUS - "OMEGA" Protocol (Grand Unified)

**Components**:
1. **Hard Negatives**: Training data augmented with emotionally-charged but non-toxic examples.
2. **Soft Labels (Perspectivist NLP)**: Label Smoothing to model annotator uncertainty.
3. **Bias Initialization**: Toxicity head starts biased towards Non-Toxic (Module 3).
4. **Progressive Freezing**: Ice Age protocol from V10.

---

In [ ]:
# Cell 1: Imports & Setup
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch.optim.lr_scheduler import OneCycleLR
from transformers import BertModel, BertTokenizer
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

In [ ]:
# Cell 2: Configuration V12 (OMEGA)
CONFIG = {
    'encoder': 'bert-base-uncased',
    'max_length': 128,
    'num_emotion_classes': 7,
    'dropout': 0.5,
    'weight_decay': 0.1,
    'batch_size': 32,
    'gradient_accumulation': 2,
    'epochs': 5,
    'lr_bert': 2e-5,
    'lr_heads': 3e-5,
    'warmup_ratio': 0.1,
    'patience': 5,
    'focal_gamma': 2.0,
    'freezing_epochs': 1,
    # V12 SPECIFIC
    'label_smoothing': 0.1,  # Perspectivist NLP: Soft Labels
}

EMO_COLS = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
DATA_DIR = '/kaggle/input/aura-v12-data'  # <-- Updated for V12 dataset with Hard Negatives
print("V12 OMEGA Loaded: Hard Negatives + Soft Labels Active 🎯")

In [ ]:
# Cell 3: AURA V12 Model (Same as V10, with Bias Init)
class AURA_CERBERUS(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.bert = BertModel.from_pretrained(config['encoder'])
        hidden = self.bert.config.hidden_size
        self.dropout = nn.Dropout(config['dropout'])
        
        self.toxicity_head = nn.Linear(hidden, 2)
        self.emotion_head = nn.Linear(hidden, config['num_emotion_classes'])
        self.sentiment_head = nn.Linear(hidden, 2)
        
        # Uncertainty parameters
        self.log_var_tox = nn.Parameter(torch.tensor(0.0))
        self.log_var_emo = nn.Parameter(torch.tensor(0.0))
        self.log_var_sent = nn.Parameter(torch.tensor(0.0))

        # Bias Initialization (Module 3 - Imbalanced Datasets)
        with torch.no_grad():
             self.toxicity_head.bias[0] = 2.5  # Non-Toxic (Majority)
             self.toxicity_head.bias[1] = -2.5 # Toxic (Minority)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = self.dropout(outputs.pooler_output)
        
        return {
            'toxicity': self.toxicity_head(pooled),
            'emotion': self.emotion_head(pooled),
            'sentiment': self.sentiment_head(pooled),
            'log_var_tox': self.log_var_tox,
            'log_var_emo': self.log_var_emo,
            'log_var_sent': self.log_var_sent
        }

print('Model AURA V12 defined (with Bias Init).')

In [ ]:
# Cell 4: Loss Functions (WITH LABEL SMOOTHING)
def focal_loss_smooth(logits, targets, gamma=2.0, weight=None, smoothing=0.0):
    """Focal Loss with optional Label Smoothing (Perspectivist NLP)."""
    # Label Smoothing: instead of [0, 1], use [smoothing/2, 1 - smoothing/2]
    ce = F.cross_entropy(logits, targets, weight=weight, reduction='none', label_smoothing=smoothing)
    p_t = torch.exp(-ce)
    return ((1 - p_t) ** gamma * ce).mean()

def kendall_loss(task_loss, log_var):
    log_var = torch.clamp(log_var, min=-5.0, max=5.0)
    precision = torch.exp(-log_var)
    return precision * task_loss + log_var

def compute_mtl_loss(outputs, batch, tox_weights, gamma, smoothing):
    total_loss = torch.tensor(0.0, device=outputs['toxicity'].device)
    
    if batch['task_mask_tox'].sum() > 0:
        # Toxicity with Focal Loss + Label Smoothing
        l = focal_loss_smooth(outputs['toxicity'][batch['task_mask_tox']], 
                              batch['tox_label'][batch['task_mask_tox']], 
                              gamma, tox_weights, smoothing)
        total_loss += kendall_loss(l, outputs['log_var_tox'])
        
    if batch['task_mask_emo'].sum() > 0:
        l = F.binary_cross_entropy_with_logits(outputs['emotion'][batch['task_mask_emo']], 
                                                batch['emo_label'][batch['task_mask_emo']])
        total_loss += kendall_loss(l, outputs['log_var_emo'])
        
    if batch['task_mask_sent'].sum() > 0:
        l = focal_loss_smooth(outputs['sentiment'][batch['task_mask_sent']], 
                              batch['sent_label'][batch['task_mask_sent']], 
                              gamma, smoothing=smoothing)  # No weights for sentiment
        total_loss += kendall_loss(l, outputs['log_var_sent'])
        
    return total_loss

print('V12 Loss defined (Focal + Label Smoothing).')

In [ ]:
# Cell 5: Datasets (Standard from V10)
class BaseDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_len):
        self.df = pd.read_csv(csv_path)
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.df)
    def encode(self, text):
        return self.tokenizer.encode_plus(str(text), max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

class ToxicityDataset(BaseDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(), 'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(int(row['label']), dtype=torch.long), 
            'emo_label': torch.zeros(len(EMO_COLS)), 
            'sent_label': torch.tensor(-1, dtype=torch.long),
            'task': 'toxicity'
        }

class EmotionDataset(BaseDataset):
    def __init__(self, csv_path, tokenizer, max_len, emo_cols):
        super().__init__(csv_path, tokenizer, max_len)
        self.emo_cols = emo_cols
        if 'label_sum' in self.df.columns: self.df = self.df[self.df['label_sum'] > 0].reset_index(drop=True)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(), 'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(-1, dtype=torch.long), 
            'emo_label': torch.tensor([float(row[c]) for c in self.emo_cols]), 
            'sent_label': torch.tensor(-1, dtype=torch.long),
            'task': 'emotion'
        }

class SentimentDataset(BaseDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(), 'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(-1, dtype=torch.long), 
            'emo_label': torch.zeros(len(EMO_COLS)), 
            'sent_label': torch.tensor(int(row['label']), dtype=torch.long),
            'task': 'sentiment'
        }

def collate_fn(batch):
    tasks = [x['task'] for x in batch]
    return {
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
        'tox_label': torch.stack([x['tox_label'] for x in batch]),
        'emo_label': torch.stack([x['emo_label'] for x in batch]),
        'sent_label': torch.stack([x['sent_label'] for x in batch]),
        'task_mask_tox': torch.tensor([t == 'toxicity' for t in tasks], dtype=torch.bool), 
        'task_mask_emo': torch.tensor([t == 'emotion' for t in tasks], dtype=torch.bool),
        'task_mask_sent': torch.tensor([t == 'sentiment' for t in tasks], dtype=torch.bool)
    }

In [ ]:
# Cell 6: Loaders (Using V12 Dataset with Hard Negatives)
tokenizer = BertTokenizer.from_pretrained(CONFIG['encoder'])

# NOTE: toxicity_train.csv now includes Hard Negatives!
tox_train = ToxicityDataset(f'{DATA_DIR}/toxicity_train.csv', tokenizer, CONFIG['max_length'])
emo_train = EmotionDataset(f'{DATA_DIR}/emotions_train.csv', tokenizer, CONFIG['max_length'], EMO_COLS)
sent_train = SentimentDataset(f'{DATA_DIR}/sentiment_train.csv', tokenizer, CONFIG['max_length'])
tox_val = ToxicityDataset(f'{DATA_DIR}/toxicity_val.csv', tokenizer, CONFIG['max_length'])

train_set = ConcatDataset([tox_train, emo_train, sent_train])
train_loader = DataLoader(train_set, batch_size=CONFIG['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader = DataLoader(tox_val, batch_size=CONFIG['batch_size'], shuffle=False, collate_fn=collate_fn)
tox_weights = torch.tensor([0.75, 1.5], device=device)

print(f"Toxicity Train (with Hard Negatives): {len(tox_train)} samples")
print("Datasets loaded.")

In [ ]:
# Cell 7: Training Engine (with Freezing)

def train_epoch(model, loader, optimizer, scheduler, config, tox_weights, epoch):
    model.train()
    
    if epoch <= config['freezing_epochs']:
        print(f"❄️ EPOCH {epoch}: ICE AGE. Backbone Frozen.")
        for param in model.bert.parameters(): param.requires_grad = False
    else:
        print(f"🔥 EPOCH {epoch}: FIRE. Backbone Unfrozen.")
        for param in model.bert.parameters(): param.requires_grad = True
            
    total_loss = 0
    optimizer.zero_grad()
    pbar = tqdm(loader, desc='Training')
    
    for step, batch in enumerate(pbar):
        for k, v in batch.items(): 
            if isinstance(v, torch.Tensor): batch[k] = v.to(device)
            
        outputs = model(batch['input_ids'], batch['attention_mask'])
        loss = compute_mtl_loss(outputs, batch, tox_weights, config['focal_gamma'], config['label_smoothing'])
        
        (loss / config['gradient_accumulation']).backward()
        
        if (step + 1) % config['gradient_accumulation'] == 0:
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item()
        if step % 50 == 0:
            σ_t = torch.exp(0.5 * model.log_var_tox).item()
            pbar.set_postfix({'loss': loss.item(), 'σ_tox': f'{σ_t:.2f}'})
            
    return total_loss / len(loader)

@torch.no_grad()
def validate(model, loader):
    model.eval()
    preds, labels = [], []
    for batch in tqdm(loader, desc='Validating', leave=False):
        ids, mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
        out = model(ids, mask)
        preds.extend(out['toxicity'].argmax(1).cpu().numpy())
        labels.extend(batch['tox_label'].to(device).cpu().numpy())
    return f1_score(labels, preds, average='macro')

In [ ]:
# Cell 8: Main Loop V12
print("="*60)
print("STARTING CERBERUS V12 OMEGA TRAINING")
print("="*60)

model = AURA_CERBERUS(CONFIG).to(device)
optimizer = torch.optim.AdamW([
    {'params': model.bert.parameters(), 'lr': CONFIG['lr_bert']},
    {'params': model.toxicity_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.emotion_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.sentiment_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': [model.log_var_tox, model.log_var_emo, model.log_var_sent], 'lr': CONFIG['lr_heads']}
], weight_decay=CONFIG['weight_decay'])

total_steps = len(train_loader) * CONFIG['epochs'] // CONFIG['gradient_accumulation']
scheduler = OneCycleLR(optimizer, max_lr=[CONFIG['lr_bert']] + [CONFIG['lr_heads']]*4, total_steps=total_steps, pct_start=CONFIG['warmup_ratio'])

best_val_f1 = 0.0
for epoch in range(1, CONFIG['epochs'] + 1):
    avg_loss = train_epoch(model, train_loader, optimizer, scheduler, CONFIG, tox_weights, epoch)
    val_f1 = validate(model, val_loader)
    σ_t = torch.exp(0.5 * model.log_var_tox).item()
    print(f'Epoch {epoch}: Loss={avg_loss:.4f}, Val F1={val_f1:.4f}, σ_tox={σ_t:.2f}')
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), 'aura_v12_omega.pt')
        print('>>> NEW BEST MODEL <<<')

print(f"\nTraining Complete. Best Val F1: {best_val_f1:.4f}")